In [1]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sn

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report

In [32]:
input_path = 'dataset/'

dataset = pd.read_csv(input_path + 'embedding_treatment_g2.csv')
dataset = dataset.drop(columns=['c'])
display(dataset.shape, dataset.head())

(580, 402)

,0,1,2,3,4,5,6,7,8,9,...,392,393,394,395,396,397,398,399,target,cls
0,-0.335372,0.217941,0.031088,-0.264938,-0.061722,0.065573,-0.420237,0.104448,0.126440,-0.037599,...,0.143072,-0.361760,0.507593,-0.038765,0.167825,0.016370,-0.049018,0.468015,<http://example/Treatment/treatment0>,decrease_effectiveness
1,-0.041577,-0.427769,0.338862,-0.266470,-0.577185,0.367611,-0.631882,-0.349803,-0.088820,0.163129,...,-0.323966,-0.154629,-0.058052,-0.258598,-0.410999,-0.118461,-0.396808,0.094013,<http://example/Treatment/treatment100>,decrease_effectiveness
2,-0.252607,-0.497979,0.169496,0.433941,0.118006,0.127568,-0.271173,-0.613200,0.066035,-0.149933,...,0.106721,-0.098707,-0.260097,0.230458,0.140243,0.380637,0.167757,0.288613,<http://example/Treatment/treatment101>,decrease_effectiveness
3,-0.321809,0.011824,0.274040,-0.058025,0.219363,0.517750,-0.276688,-0.408989,-0.295073,-0.079781,...,0.160993,-0.473746,0.228025,0.239765,0.340143,-0.056037,-0.183916,0.622012,<http://example/Treatment/treatment102>,decrease_effectiveness
4,0.348819,0.233717,-0.097943,0.036843,0.217736,-0.121205,0.118148,-0.448963,-0.100242,0.073976,...,-0.333424,-0.377835,-0.128795,-0.505504,-0.142017,-0.080079,0.283112,-0.067669,<http://example/Treatment/treatment103>,decrease_effectiveness


In [33]:
dataset.loc[dataset.cls=='decrease_effectiveness', 'class'] = 1
dataset.loc[dataset.cls=='effective', 'class'] = 0

In [34]:
target_class = dataset[['target', 'class']]
data = dataset.drop(['cls', 'class'], axis=1)

display(target_class.shape, target_class.head(2), data.shape, data.head(2))

(580, 2)

,target,class
0,<http://example/Treatment/treatment0>,1.0
1,<http://example/Treatment/treatment100>,1.0


(580, 401)

,0,1,2,3,4,5,6,7,8,9,...,391,392,393,394,395,396,397,398,399,target
0,-0.335372,0.217941,0.031088,-0.264938,-0.061722,0.065573,-0.420237,0.104448,0.12644,-0.037599,...,-0.275158,0.143072,-0.361760,0.507593,-0.038765,0.167825,0.016370,-0.049018,0.468015,<http://example/Treatment/treatment0>
1,-0.041577,-0.427769,0.338862,-0.266470,-0.577185,0.367611,-0.631882,-0.349803,-0.08882,0.163129,...,0.006792,-0.323966,-0.154629,-0.058052,-0.258598,-0.410999,-0.118461,-0.396808,0.094013,<http://example/Treatment/treatment100>


In [35]:
target_class['class'].value_counts()

1.0    524
0.0     56
Name: class, dtype: int64

In [36]:
new_data = data.drop(columns=['target'])

In [37]:
X_train, X_test, y_train, y_test = train_test_split(new_data.values, target_class['class'].values, 
                                                    test_size=0.25, random_state=42)

In [44]:
feature_names = new_data.columns
parameters = {"max_depth": range(2,10)}
#clf = tree.DecisionTreeClassifier(random_state=41)
clf = RandomForestClassifier(random_state=0)
grid = GridSearchCV(estimator=clf,param_grid=parameters)
grid_res = grid.fit(X_train, y_train)
best_clf = grid_res.best_estimator_


#predictions = (clf.fit(X_train, y_train)).predict(X_test)
best_clf.score(X_test, y_test)
y_pred = best_clf.predict(X_test)

In [45]:
target_names = ['Effective treatment','Low-effect treatment']
print (classification_report(y_test, y_pred, target_names=target_names))

                      precision    recall  f1-score   support

 Effective treatment       1.00      0.11      0.20         9
Low-effect treatment       0.94      1.00      0.97       136

            accuracy                           0.94       145
           macro avg       0.97      0.56      0.59       145
        weighted avg       0.95      0.94      0.92       145



In [108]:
from dtreeviz.trees import dtreeviz
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'
viz = dtreeviz(best_clf[0], new_data, target_class['class'], target_name='class', 
               feature_names=feature_names, class_names=target_names, fancy=False, 
               show_root_edge_labels=True)
viz.view()

In [30]:
from sklearn import svm

clf = svm.SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [31]:
target_names = ['Effective treatment','Low-effect treatment']
print (classification_report(y_test, y_pred, target_names=target_names))

                      precision    recall  f1-score   support

 Effective treatment       1.00      0.56      0.71         9
Low-effect treatment       0.97      1.00      0.99       136

            accuracy                           0.97       145
           macro avg       0.99      0.78      0.85       145
        weighted avg       0.97      0.97      0.97       145

